## Data Sceince Capstone Project 2

##### Hello Everyone,
#### This project aims to cluster Jordanian major Neighborhoods
#### As my country is relatively small, i tried to cluster the major ones

### The aim is find a suitable area to Open a Cafe, the approch is to cluster Neighborhoods then find the clusters that are popular with Cafe's, then choose a Neighborhood in this cluster with the lower number of Cafe's to be a target area

In [13]:
import pandas as pd
import numpy as np
import lxml.html as lh
import requests
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

##### Loading the data with coordinates from Excel file that I prepared

In [3]:
jordan_df=pd.read_excel('jordan_qada.xlsx')

In [9]:
jordan_df.rename(columns={"Qadaa":"Neighborhood"},inplace="True")

In [10]:
jordan_df.head()

,Neighborhood,latitude,longitude
0,Arda,32.143158,35.710843
1,Athroh,30.379695,35.646346
2,Erhab,32.314655,36.037004
3,Azraq,31.816432,36.774597
4,Jafer,30.018754,36.186590


In [12]:
address = 'Jordan'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Jordan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Jordan are 31.1667049, 36.941628.


In [20]:
map_jordan = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, label in zip(jordan_df['latitude'], jordan_df['longitude'], jordan_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jordan)  
    
map_jordan

In [21]:
CLIENT_ID = 'OTGZXJNFTNL5CI0FLOFKHVG4DZHKOMZL2GWUFAGBOKBIXTV3' # your Foursquare ID
CLIENT_SECRET = 'T0F0WU2E3I413GVFZWQQIS1KSDUZCAA1RXL1I1Z5YDOXFOP4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
jordan_venues = getNearbyVenues(names=jordan_df['Neighborhood'],
                                   latitudes=jordan_df['latitude'],
                                   longitudes=jordan_df['longitude']
                                  )


Arda
Athroh
Erhab
Azraq
Jafer
Khaldieh
Deseh
Dhulail
Areed
Faisalieh
Mregha
Mastabeh
Manshyeh
Mujeb
Um_Basateen
Um_Jmaal
Um_Rsas
Um_Qutain
Eil
Borma
Balama
Berien
Jreneh
Husban
Housha
Deir_Kahf
Rujm Shami
Zai
Sama Sarhan
Sabha
Sakhra
Arjan
Yarqa
Ghour Mazraa
Maeen
Mleh
Moaab
Wadi_araba
South_Aghwar
North_Aghwar
North_Badyeh
Salhieh
Petra
Jamaa
Jezeh
Hasa
Huseinieh
Rusaifeh
Ramtha
Rowaished
Shobak
Shoneh
Taibeh
Faqooa
Qaser
Qatraneh
Qweira
Quesmieh
Kura
South_mazar
North_Mazar
Mowaqar
Hashmieh
Wasatieh
Bseera
Bani_Abeid
Bani_Kenanah
Dair_alla
Dheiban
Sahab
Aay
Ain_Basha
Center_Irbid
Center_zarqa
Center_Salt
Center_Tafielah
Center_Aqaba
Center_Karak
Center_Mafraq
Center_Jarash
Center_Ajloun
Center_Amman
Center_Madaba
Center_Maan
Kufranjah
Fuhais
Marka
Naoor
Wadi_seer


In [45]:
jordan_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arda,32.143158,35.710843,Sunset cafe,32.103053,35.702953,Café
1,Arda,32.143158,35.710843,Zai - AlSalt,32.102171,35.718854,Nature Preserve
2,Arda,32.143158,35.710843,Subeihi الصبيحي,32.132712,35.700863,Other Great Outdoors
3,Arda,32.143158,35.710843,Zay National Park,32.107079,35.705137,Park
4,Erhab,32.314655,36.037004,The Farm,32.323427,36.046546,Farm
5,Erhab,32.314655,36.037004,مزرعة محمد موسى الوحش (ابو مي),32.331108,36.049734,Farm
6,Erhab,32.314655,36.037004,Tamimi's farm,32.322508,35.987564,Farm
7,Azraq,31.816432,36.774597,"MS Air Base, Jordan",31.833032,36.794555,Airport
8,Azraq,31.816432,36.774597,Azraq Lodge,31.836030,36.807627,Motel
9,Azraq,31.816432,36.774597,Azraq Water Reserve,31.834314,36.820154,Lake


In [49]:
jordan_venues[['Venue Category']] = jordan_venues[['Venue Category']].replace('Neighborhood','Neighborhood#', regex=True)

In [50]:
jordan_venues.shape

(1013, 7)

In [51]:
venues_tp=pd.get_dummies(jordan_venues['Venue Category'])

In [52]:
venues_tp

,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Art Museum,...,Sushi Restaurant,Syrian Restaurant,Tea Room,Theater,Tourist Information Center,Trail,Turkish Restaurant,Watch Shop,Waterfront,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
venues_tp['Neighborhood']=jordan_venues['Neighborhood']

In [54]:
fixed_columns = [venues_tp.columns[-1]] + list(venues_tp.columns[:-1])
venues_tp = venues_tp[fixed_columns]

In [57]:
venues_tp.head(10)

,Neighborhood,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Art Gallery,...,Sushi Restaurant,Syrian Restaurant,Tea Room,Theater,Tourist Information Center,Trail,Turkish Restaurant,Watch Shop,Waterfront,Wings Joint
0,Arda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Erhab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Erhab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Erhab,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Azraq,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Azraq,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Azraq,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
venues_grouped=venues_tp.groupby('Neighborhood').mean().reset_index()

In [70]:
venues_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Art Gallery,...,Sushi Restaurant,Syrian Restaurant,Tea Room,Theater,Tourist Information Center,Trail,Turkish Restaurant,Watch Shop,Waterfront,Wings Joint
0,Ain_Basha,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Arda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Areed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arjan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Azraq,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
topx=5
for hood in venues_grouped['Neighborhood']:
    print("----------"+hood+"------------")
    temp=venues_grouped[venues_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['Neighborhood','freq']
    temp=temp.iloc[1:]
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(topx))
    print('\n')
    

----------Ain_Basha------------
                Neighborhood       freq
0  Middle Eastern Restaurant   0.111111
1                       Café   0.111111
2                Pizza Place  0.0740741
3                        Gym  0.0740741
4         Falafel Restaurant  0.0740741


----------Arda------------
           Neighborhood  freq
0                  Park  0.25
1  Other Great Outdoors  0.25
2                  Café  0.25
3       Nature Preserve  0.25
4     Afghan Restaurant     0


----------Areed------------
        Neighborhood      freq
0     Scenic Lookout  0.333333
1       Carpet Store  0.333333
2         Hot Spring  0.333333
3  Afghan Restaurant         0
4             Museum         0


----------Arjan------------
          Neighborhood freq
0      Nature Preserve  0.2
1               Forest  0.2
2           Campground  0.2
3                 Café  0.2
4  Arts & Crafts Store  0.2


----------Azraq------------
                Neighborhood freq
0       Gym / Fitness Center  0.2
1      

          Neighborhood freq
0           Restaurant    1
1    Afghan Restaurant    0
2               Museum    0
3  Monument / Landmark    0
4                Motel    0


----------Naoor------------
                Neighborhood     freq
0  Middle Eastern Restaurant  0.15625
1                      Plaza  0.09375
2       Fast Food Restaurant  0.09375
3               Burger Joint   0.0625
4        American Restaurant   0.0625


----------North_Aghwar------------
        Neighborhood  freq
0    Border Crossing  0.25
1               Park  0.25
2               Farm  0.25
3     Scenic Lookout  0.25
4  Afghan Restaurant     0


----------North_Badyeh------------
        Neighborhood      freq
0        Coffee Shop  0.333333
1         Watch Shop  0.333333
2             Bakery  0.333333
3  Afghan Restaurant         0
4             Museum         0


----------North_Mazar------------
          Neighborhood freq
0     Asian Restaurant    1
1    Afghan Restaurant    0
2  Monument / Landmark    0
3   

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ain_Basha,Middle Eastern Restaurant,Café,Grocery Store,Gym,Falafel Restaurant,Mountain,Pizza Place,Gym / Fitness Center,Shopping Mall,Food & Drink Shop
1,Arda,Park,Other Great Outdoors,Café,Nature Preserve,Department Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Doner Restaurant,Diner
2,Areed,Scenic Lookout,Carpet Store,Hot Spring,Wings Joint,Dessert Shop,Falafel Restaurant,Eastern European Restaurant,Duty-free Shop,Donut Shop,Doner Restaurant
3,Arjan,Arts & Crafts Store,Campground,Café,Nature Preserve,Forest,Dessert Shop,Falafel Restaurant,Eastern European Restaurant,Duty-free Shop,Donut Shop
4,Azraq,Gym / Fitness Center,Airport,Lake,Middle Eastern Restaurant,Motel,Diner,Falafel Restaurant,Eastern European Restaurant,Duty-free Shop,Donut Shop


In [101]:
cluster_no=5
venues_grouped_clustering=venues_grouped.drop('Neighborhood',1)
kmeans=KMeans(n_clusters=cluster_no,random_state=0).fit(venues_grouped_clustering)
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 1, 3, 1, 1, 1])

In [121]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

jordan_merged = jordan_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
jordan_merged = jordan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

jordan_merged.head() 

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arda,32.143158,35.710843,3.0,Park,Other Great Outdoors,Café,Nature Preserve,Department Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Doner Restaurant,Diner
1,Athroh,30.379695,35.646346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Erhab,32.314655,36.037004,1.0,Farm,Wings Joint,Gym,Falafel Restaurant,Eastern European Restaurant,Duty-free Shop,Donut Shop,Doner Restaurant,Diner,Dessert Shop
3,Azraq,31.816432,36.774597,3.0,Gym / Fitness Center,Airport,Lake,Middle Eastern Restaurant,Motel,Diner,Falafel Restaurant,Eastern European Restaurant,Duty-free Shop,Donut Shop
4,Jafer,30.018754,36.186590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
jordan_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

In [123]:
jordan_merged['Cluster Labels']=jordan_merged['Cluster Labels'].astype(int)

In [116]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(cluster_no)
ys = [i + x + (i*x)**2 for i in range(cluster_no)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jordan_merged['latitude'], jordan_merged['longitude'], jordan_merged['Neighborhood'], jordan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [125]:
jordan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arda,32.143158,35.710843,Sunset cafe,32.103053,35.702953,Café
1,Arda,32.143158,35.710843,Zai - AlSalt,32.102171,35.718854,Nature Preserve
2,Arda,32.143158,35.710843,Subeihi الصبيحي,32.132712,35.700863,Other Great Outdoors
3,Arda,32.143158,35.710843,Zay National Park,32.107079,35.705137,Park
4,Erhab,32.314655,36.037004,The Farm,32.323427,36.046546,Farm


#### Finding the Cluster with most Cafe's

In [146]:
jordan_merged_2=jordan_merged.iloc[:,[0,3]]

In [156]:
jordan_venue_cluster=pd.merge(jordan_venues,jordan_merged_2)

In [161]:
jordan_venue_cluster_cafe=jordan_venue_cluster[jordan_venue_cluster['Venue Category']=='Café']

In [164]:
jordan_venue_cluster_cafe.groupby('Cluster Labels').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Cluster Labels,,,,,,,
1,2,2,2,2,2,2,2
3,86,86,86,86,86,86,86


#### Neighborhoods in Cluster 3 are popular with Cafe's

#### Let us see which Neighborhood in Cluster 3 is having low number of Cafe's to be the target area

In [181]:
jordan_venue_cluster_cafe_target=jordan_venue_cluster_cafe[jordan_venue_cluster_cafe['Cluster Labels']==3].groupby('Neighborhood').count().sort_values('Venue')

## Target Areas to open Cafe

In [184]:
jordan_venue_cluster_cafe_target=jordan_venue_cluster_cafe_target.iloc[0:5,0]

In [188]:
jordan_venue_cluster_cafe_target

Neighborhood
Yarqa           1
Arda            1
Arjan           1
Center_Aqaba    1
Petra           1
Name: Neighborhood Latitude, dtype: int64